In [1]:
!pip install ipyparallel

In [2]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib as plot
import seaborn as sns
import warnings
from patsy import dmatrices
%pylab inline
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\magics\pylab.py:159: UserWarning: pylab import has clobbered these variables: ['plot']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [3]:
df = pd.read_csv('./data/crime_test.csv')

In [4]:
#droping x and y coordinates since we have latitude and logitude already
df.index = pd.DatetimeIndex(df.Date)
df['Month'] = df.index.month
df['Day'] = df.index.day
df = df.drop(['Date','Description','Location', 'X Coordinate', 'Y Coordinate', 'COMMUNITY AREA NAME'], axis = 1).drop_duplicates()


In [5]:
df.dtypes

Primary Type                                     object
Location Description                             object
Arrest                                             bool
Domestic                                           bool
Beat                                              int64
District                                        float64
Ward                                            float64
Community Area                                    int64
Year                                              int64
Latitude                                        float64
Longitude                                       float64
PERCENT OF HOUSING CROWDED                      float64
PERCENT HOUSEHOLDS BELOW POVERTY                float64
PERCENT AGED 16+ UNEMPLOYED                     float64
PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA    float64
PERCENT AGED UNDER 18 OR OVER 64                float64
PER CAPITA INCOME                               float64
HARDSHIP INDEX                                  

In [6]:
# transform into categorical 
for col in df.columns:
    if col == 'Longitude':
        
        print('none')
    elif col == 'Latitude':
        print('none')
    else:
        df[col] = pd.Categorical(df[col])

none
none


## Logistics Regression

In [7]:
df.columns

Index(['Primary Type', 'Location Description', 'Arrest', 'Domestic', 'Beat',
       'District', 'Ward', 'Community Area', 'Year', 'Latitude', 'Longitude',
       'PERCENT OF HOUSING CROWDED', 'PERCENT HOUSEHOLDS BELOW POVERTY',
       'PERCENT AGED 16+ UNEMPLOYED',
       'PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA',
       'PERCENT AGED UNDER 18 OR OVER 64', 'PER CAPITA INCOME ',
       'HARDSHIP INDEX', 'Month', 'Day'],
      dtype='object')

In [8]:
df_model = df
df_model.rename(columns={'PERCENT OF HOUSING CROWDED':'percent_housing_crowded'}, inplace=True)
df_model.rename(columns={'PERCENT HOUSEHOLDS BELOW POVERTY':'percent_below_poverty'}, inplace=True)
df_model.rename(columns={'PERCENT AGED 16+ UNEMPLOYED':'percent_above_16_unemployed'}, inplace=True)
df_model.rename(columns={'PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA':'percent_above_25_no_diploma'}, inplace=True)
df_model.rename(columns={'PERCENT AGED UNDER 18 OR OVER 64':'percent_old_young'}, inplace=True)
df_model.rename(columns={'PER CAPITA INCOME ':'per_capita_income'}, inplace=True)
df_model.rename(columns={'HARDSHIP INDEX':'hardship_index'}, inplace=True)
df_model.rename(columns={'Location Description':'location_description'}, inplace=True)
df_model.rename(columns={'Community Area':'community_area'}, inplace=True)
df_model.rename(columns={'Primary Type':'primary_type'}, inplace=True)

In [9]:
Y, X = dmatrices('primary_type ~ 0  + Month + Day + location_description + District + Ward + Latitude + Longitude + percent_housing_crowded + percent_below_poverty + percent_above_16_unemployed + percent_above_25_no_diploma + percent_old_young + per_capita_income + hardship_index', df_model, return_type = 'dataframe')

In [11]:
y_theft = Y['primary_type[THEFT]'].values


In [12]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LogisticRegression

In [13]:
X_train, X_test_theft, y_train_theft, y_test_theft = train_test_split(X, y_theft, test_size=0.3, random_state=1)

In [14]:
log_model = LogisticRegression()

In [15]:
log_crime = log_model.fit(X_train, y_train_theft)

In [16]:
log_pred_theft = log_model.predict(X_test_theft)
print('Accuracy:',metrics.accuracy_score(y_test_theft,log_pred_theft))

##checking weight of the variables:

log_weight = Series(log_model.coef_[0],index=X.columns.values)

print('\n Features with the most NEGATIVE weights:\n')
    
for i in log_weight.sort_values()[:10].index:
    print(i)
    
    
print('\n Features with the most POSITIVE weights:\n')
    
for i in log_weight.sort_values()[-10:].index:
    print(i)

Accuracy: 0.7980943784163281

 Features with the most NEGATIVE weights:

location_description[T.SIDEWALK]
location_description[T.ALLEY]
location_description[T.APARTMENT]
location_description[T.RESIDENCE-GARAGE]
location_description[T.RESIDENCE]
location_description[T.CHA PARKING LOT/GROUNDS]
Ward[T.26.0]
District[T.11.0]
location_description[T.CTA PLATFORM]
location_description[T.BANK]

 Features with the most POSITIVE weights:

location_description[T.VEHICLE NON-COMMERCIAL]
location_description[T.CONVENIENCE STORE]
Ward[T.42.0]
location_description[T.RESIDENTIAL YARD (FRONT/BACK)]
location_description[T.CTA TRAIN]
location_description[T.ATHLETIC CLUB]
location_description[T.DRUG STORE]
location_description[T.SMALL RETAIL STORE]
location_description[T.GROCERY FOOD STORE]
location_description[T.DEPARTMENT STORE]


In [17]:
log_weight.sort_values(ascending=False)[-10:]

location_description[T.BANK]                      -0.338642
location_description[T.CTA PLATFORM]              -0.339429
District[T.11.0]                                  -0.360413
Ward[T.26.0]                                      -0.411736
location_description[T.CHA PARKING LOT/GROUNDS]   -0.416921
location_description[T.RESIDENCE]                 -0.777286
location_description[T.RESIDENCE-GARAGE]          -0.965853
location_description[T.APARTMENT]                 -1.028572
location_description[T.ALLEY]                     -1.204964
location_description[T.SIDEWALK]                  -1.366094
dtype: float64

In [18]:
log_weight.sort_values(ascending=False)[:10]

location_description[T.DEPARTMENT STORE]                 2.170502
location_description[T.GROCERY FOOD STORE]               1.596241
location_description[T.SMALL RETAIL STORE]               1.324423
location_description[T.DRUG STORE]                       0.708858
location_description[T.ATHLETIC CLUB]                    0.499147
location_description[T.CTA TRAIN]                        0.488374
location_description[T.RESIDENTIAL YARD (FRONT/BACK)]    0.452982
Ward[T.42.0]                                             0.363881
location_description[T.CONVENIENCE STORE]                0.351235
location_description[T.VEHICLE NON-COMMERCIAL]           0.292782
dtype: float64